In [1]:
import os
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

import dotenv
dotenv.load_dotenv()

client = OpenAI(
  api_key=os.environ.get("OPENAI_API_KEY"),
)

In [61]:
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [62]:
from sklearn.metrics.pairwise import cosine_similarity
def get_distance(text1, text2, model="text-embedding-3-small"):
   text1 = str(text1).lower()
   text2 = str(text2).lower()
   embedding1 = get_embedding(text1, model)
   embedding2 = get_embedding(text2, model)
   return cosine_similarity([embedding1], [embedding2])[0][0]

In [72]:
# go through sample.json and ask questions to gpt
import json
all_qs = json.load(open("filtered_train.json"))

# randomly sample 0 questions
import random
sample = random.sample(all_qs, 60)

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "",
        }
    ],
    model="gpt-4o-mini-2024-07-18",
    # temperature=0.2,
    

)

str(chat_completion.choices[0].message.content)


In [ ]:
output = []

sum_1 = 0
sum_2 = 0

for i in sample:
    # print("-"*20)
    ootb_llm_rseponse = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Answer the question as concisely as possible with ONLY one answer without any other text:  {i['nq_question']}",
            }
        ],
        model="gpt-4o-mini-2024-07-18"
    )
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Rewrite this question replacing all questions with a what, but retain the meaning. The Question: {i['nq_question']}",
            }
        ],
        model="gpt-4o-mini-2024-07-18"
    )

    print(chat_completion.choices[0].message.content)


    final_answer = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Answer the question as concisely as possible with ONLY one answer without any other text:  {chat_completion.choices[0].message.content}",
            }
        ],
        model="gpt-4o-mini-2024-07-18"
    )


    ambig_answer_distance = 0
    disambig_answer_distance = 0
    for ans in i["nq_answer"]:
        curr_ambig_answer_distance = get_distance(ans, ootb_llm_rseponse.choices[0].message.content)
        curr_disambig_answer_distance = get_distance(ans, final_answer.choices[0].message.content)
        if curr_ambig_answer_distance > ambig_answer_distance:
            ambig_answer_distance = curr_ambig_answer_distance
        if curr_disambig_answer_distance > disambig_answer_distance:
            disambig_answer_distance = curr_disambig_answer_distance

    curr = {
        "data_id": i["nq_id"],
        "ambig_question": i["nq_question"],
        "ambig_prompt_response": ootb_llm_rseponse.choices[0].message.content,
        "disambig_question": chat_completion.choices[0].message.content,
        "disambig_prompt_response": final_answer.choices[0].message.content,
        "ground_truth": i["nq_answer"],
        "question_distance": get_distance(i["nq_question"], chat_completion.choices[0].message.content),
        "answer_distance": get_distance(ootb_llm_rseponse.choices[0].message.content, final_answer.choices[0].message.content),
        "ambig_answer_distance": ambig_answer_distance,
        "disambig_answer_distance": disambig_answer_distance
    }

    sum_1 += curr["ambig_answer_distance"]
    sum_2 += curr["disambig_answer_distance"]

    # print(f"question_distance: {get_distance(i["nq_question"], chat_completion.choices[0].message.content)}")
    # print(f"answer_distance: {get_distance(ootb_llm_rseponse.choices[0].message.content, final_answer.choices[0].message.content)}")
    # if i["nq_answer"] is list:
    # print(f"ambig_answer_distance: {get_distance(i["nq_answer"], ootb_llm_rseponse.choices[0].message.content)}")
    # print(f"disambig_answer_distance: {get_distance(i["nq_answer"], final_answer.choices[0].message.content)}")

    # output.append(curr)

    # save output to json file
    # json.dump(output, open("disambig_prompt_4o_mini.json", "w"))

    print(curr)

print("="*20)
print(f"Average ambig answer distance: {sum_1/len(sample)}")
print(f"Average disambig answer distance: {sum_2/len(sample)}")


In [ ]:
# go through sample.json and ask questions to gpt
import json
all_qs = json.load(open("filtered_train.json"))

# randomly sample 0 questions
import random
sample2 = random.sample(all_qs, 100)

input = []

for i in sample2:
    if len(i["nq_answer"]) > 1:
        ootb_llm_rseponse = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"Answer the question as concisely as possible with ONLY one answer without any other text:  {i['nq_question']}",
                }
            ],
            model="gpt-4o-mini-2024-07-18"
        )
        closest_ambig_answer = ""
        ambig_answer_distance = 0
        for ans in i["nq_answer"]:
            curr_ambig_answer_distance = get_distance(ans, ootb_llm_rseponse.choices[0].message.content)
            if curr_ambig_answer_distance > ambig_answer_distance:
                ambig_answer_distance = curr_ambig_answer_distance
                closest_ambig_answer = ans

        print(f"Question: {i['nq_question']}")
        print(f"LLM response: {ootb_llm_rseponse.choices[0].message.content}")
        print(f"Closest ambig answer: {closest_ambig_answer}")
        print(f"Other answers: {i['nq_answer']}")
        print("="*20)
    else:
        closest_ambig_answer = i["nq_answer"][0]

    curr_i = {
        "messages": [
            {
                "role": "user",
                "content": i["nq_question"],
            },
            {
                "role": "assistant",
                "content": closest_ambig_answer
            }
        ]
    }

    print(curr_i)

    # add curri to disambig_prompt_4o_mini_input.jsonl 
    # every line is a json object
    json.dump(curr_i, open("disambig_prompt_4o_mini_input.jsonl", "a"))
    # add a newline character
    open("disambig_prompt_4o_mini_input.jsonl", "a").write("\n")
    

In [ ]:
from openai import OpenAI
client = OpenAI()

out = client.files.create(
  file=open("disambig_prompt_4o_mini_input.jsonl", "rb"),
  purpose="fine-tune"
)

print(out)

client.fine_tuning.jobs.create(
  model="gpt-4o-mini-2024-07-18",
  training_file="disambig_prompt_4o_mini_input.jsonl",
)

In [ ]:
client.fine_tuning.jobs.create(
  model="gpt-4o-mini-2024-07-18",
  training_file="file-93u9IRYgMPCYNlCa86tViZQa",
)

In [ ]:
job = str(client.fine_tuning.jobs.retrieve("ftjob-vKbrLHd71B1tgCdyLxFDJ1LX"))

print(job)

# print after status=\' till next \'
print(job[job.index("status='")+len("status='"):])

In [73]:
model = "ft:gpt-4o-mini-2024-07-18:asu::AJRC7sIy:ckpt-step-100"
sum_ootb = 0
sum_fine_tuned = 0

for i in sample:
    print("Question: ", i["nq_question"])

    ootb_llm_rseponse = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Answer the question as concisely as possible with ONLY one answer without any other text:  {i['nq_question']}",
            }
        ],
        model="gpt-4o-mini-2024-07-18"
    )

    print(ootb_llm_rseponse.choices[0].message.content)
    fine_tuned_response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Answer the question as concisely as possible with ONLY one answer without any other text:  {i['nq_question']}",
            }
        ],
        model=model
    )


    print(fine_tuned_response.choices[0].message.content)

    ambig_answer_distance = 0
    fine_tuned_answer_distance = 0
    for ans in i["nq_answer"]:
        # print(ans)
        curr_ambig_answer_distance = get_distance(ans, ootb_llm_rseponse.choices[0].message.content)
        ft_answer_distance = get_distance(ans, fine_tuned_response.choices[0].message.content)
        if curr_ambig_answer_distance > ambig_answer_distance:
            ambig_answer_distance = curr_ambig_answer_distance

        if ft_answer_distance > fine_tuned_answer_distance:
            fine_tuned_answer_distance = ft_answer_distance

    print([ambig_answer_distance, fine_tuned_answer_distance])

    sum_ootb += ambig_answer_distance
    sum_fine_tuned += fine_tuned_answer_distance

    print("Ground truth: ", i["nq_answer"])
    print("="*20)

print("="*20)
print(f"Average ambig answer distance: {sum_ootb/len(sample)}")
print(f"Average fine-tuned answer distance: {sum_fine_tuned/len(sample)}")


Question:  Who sings tis so sweet to trust in jesus with alan jackson?
Tis So Sweet to Trust in Jesus is sung by Alan Jackson.
Isaac Watts
[0.34217410585028885, 0.20998489049939312]
Ground truth:  ['Dani Jackson']
Question:  In which language is the indian national anthem?
Hindi.
Hindi
[0.5157684433673098, 0.5129184151070041]
Ground truth:  ['Bengali']
Question:  Girl who played christine in phantom of the opera?
Emmy Rossum
Emmy Rossum
[1.0000000000000009, 1.0000000000000009]
Ground truth:  ['Emmy Rossum']
Question:  Where do the sacramento kings play at home?
Golden 1 Center.
Golden 1 Center
[0.7862183186106912, 0.7961331293689771]
Ground truth:  ['at the Golden 1 Center']
Question:  The italian term for a loud volume level is?
Forte
forte
[0.9999999999999993, 0.9999993429175672]
Ground truth:  ['Forte']
Question:  Where did the saying with great power comes great responsibility come from?
Spider-Man comic books.
Voltaire
[0.29094322016704743, 0.18751326143868327]
Ground truth:  ['Am